In [1]:
import json 
import pandas as pd 
import numpy as np
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#import TS files
f1 = 'https://upday-data-assignment.s3.eu-west-1.amazonaws.com/lake/2019-02-15.tsv'
f2 = 'https://upday-data-assignment.s3.eu-west-1.amazonaws.com/lake/2019-02-16.tsv'
f3 = 'https://upday-data-assignment.s3.eu-west-1.amazonaws.com/lake/2019-02-17.tsv'

# Convert to dataframes
t1 = pd.read_csv(f1, sep='\t', header=0)
t2 = pd.read_csv(f1, sep='\t', header=0)
t3 = pd.read_csv(f1, sep='\t', header=0)

In [3]:
#Apped all files into 1
T = t1.append(t2.append(t3)).reset_index()

#Create a copy for test
T_1 = T.copy()

In [6]:
# replace nan values
T_1['ATTRIBUTES'] = T_1['ATTRIBUTES'].fillna('{}')
T_1['ATTRIBUTES'] = T_1['ATTRIBUTES'].apply(json.loads)

In [8]:
# Flatten json and append columns to main DF
df_ATTRIBUTES = pd.json_normalize(T_1['ATTRIBUTES'])
complete_df = pd.concat([T_1, df_ATTRIBUTES], axis=1).drop('ATTRIBUTES', axis=1)

In [10]:
#prepare data for table article_performance 
stage = complete_df[['id', 'ID', 'TIMESTAMP', 'EVENT_NAME','title', 'category']][(complete_df.EVENT_NAME.str.contains("card")) | (complete_df.EVENT_NAME.str.contains("article"))].sort_values(by=['id']).reset_index(drop=True)

#limit the timestamp to only date part
stage['date'] = stage.TIMESTAMP.str[:10]

## Prepare data for article_performance table 

In [11]:
# count rows for events
art_perf = stage.groupby(['id', 'date','title', 'category', 'EVENT_NAME']
                ).size().unstack().reset_index().fillna(0)

# combine numbers for card view events
art_perf['card_views']=art_perf.my_news_card_viewed+art_perf.top_news_card_viewed

In [12]:
#create dataframe formatted for article_performance table
article_performance = art_perf.drop(['my_news_card_viewed', 'top_news_card_viewed'], axis=1).rename(columns={"article_viewed": "article_views", "id":"article_id"})

# reoder columns as per requirement
article_performance = article_performance[['article_id', 'date', 'title', 'category', 'card_views', 'article_views']]

In [14]:
article_performance.head()

EVENT_NAME,article_id,date,title,category,card_views,article_views
0,-1e_PWmoy-tnoBrgcDA2-A,2019-02-15,"Aktorzy, którzy zginęli na planie filmowym i p...",fashion_beauty_lifestyle,3.0,0.0
1,-1kNxqvvZsmYDKJ5C-SGGQ,2019-02-15,Mort de Kristoff St. John : son ex-femme dévoi...,people_shows,18.0,0.0
2,-36YHd9KZ1RUVBfCXkvITg,2019-02-15,This Crazy-Affordable Skin Care Brand Gave Me ...,fashion_beauty_lifestyle,9.0,0.0
3,-3RsLiL0PwPUeEwwrEbIMw,2019-02-15,"Lazio, ti è andata pure bene! Banega sontuoso:...",sports,0.0,3.0
4,-72o6DS38oeYSCfz5I5I7Q,2019-02-15,"Zurigo-Napoli, scontri tra ultras nel pre-part...",sports,3.0,0.0


## Prepare data for user_performance table 

##### The id column for user was not clear. 
The "ID" column contained only 16 unique ids which accounted to few card views. They also did not have any article views.
Due to this I continued using the "id" column instead to calculate the ctr.


In [16]:
# prepare data for user performance table
usr_perf = article_performance[['article_id', 'date', 'card_views', 'article_views']]

# combine numbers for card view events
usr_perf['ctr']=usr_perf.article_views / usr_perf.card_views

#create dataframe formatted for user_performance table
user_performance  = usr_perf.drop(['article_views', 'card_views'], axis=1).rename(columns={"article_id": "user_id"}).reset_index(drop=True)

<ipython-input-16-aeb077de32e8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usr_perf['ctr']=usr_perf.article_views / usr_perf.card_views


In [17]:
user_performance.head()

EVENT_NAME,user_id,date,ctr
0,-1e_PWmoy-tnoBrgcDA2-A,2019-02-15,0.0
1,-1kNxqvvZsmYDKJ5C-SGGQ,2019-02-15,0.0
2,-36YHd9KZ1RUVBfCXkvITg,2019-02-15,0.0
3,-3RsLiL0PwPUeEwwrEbIMw,2019-02-15,inf
4,-72o6DS38oeYSCfz5I5I7Q,2019-02-15,0.0
